In [3]:
! curl "http://zero.borm.org:8000"

^C


In [1]:
!pip install PyVCF
import vcf

  Retrying (Retry(total=4, connect=None, read=None, redirect=None)) after connection broken by 'NewConnectionError('<pip._vendor.requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7fdf9077aba8>: Failed to establish a new connection: [Errno 101] Network is unreachable',)': /simple/pyvcf/


# REPLACE pjotr.me with your name!!

In [1]:

print("@prefix : <http://emmy.me/rdf/ns#> .\n")

vcf_reader = vcf.Reader(open('/home/user/gatk_exome.vcf', 'r'))
count=0
for record in vcf_reader:
    name = "chr"+record.CHROM+"_"+str(record.POS)+"_"+str(record.ALT[0])
    print(":"+name)
    print("  :query_id \""+name+"\" ;")
    print("  :chr \""+record.CHROM+"\" ;")
    print("  :pos \""+str(record.POS)+"\" ;")
    print("  :alt \""+str(record.ALT[0])+"\" .")
    count +=1
    if count > 5:
        break


@prefix : <http://emmy.me/rdf/ns#> .



NameError: name 'vcf' is not defined

### Now write the same to file with samples and a few extra triples

In [36]:

import vcf

buf = """
@prefix    :   <http://pjotr.me/rdf/ns#> .
@prefix rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .

"""

vcf_reader = vcf.Reader(open('/home/user/gatk_exome.vcf', 'r'))
for record in vcf_reader:
    name = "chr"+record.CHROM+"_"+str(record.POS)+"_"+str(record.ALT[0])
    buf += ":"+name+"\n"
    buf += "  :variant \""+name+"\" ;\n"
    buf += "  rdf:type :snp ;\n"
    buf += "  :chr \""+record.CHROM+"\" ;\n"
    buf += "  :pos \""+str(record.POS)+"\" ;\n"
    buf += "  :alt \""+str(record.ALT[0])+"\" .\n"
    for sample in record.samples:
        buf += ":"+name+"_"+str(sample.sample)+"\n"
        if sample["DP"]:
            buf += "  :dp   "+str(sample["DP"])+" ;\n"
        buf += "  :vcf :"+name+" .\n"

# Finally we connect a few SNPs

buf += """
:chr1_66058513_G  :controls  :chr1_114354942_G .
:chr1_114354942_G :controls  :chr1_144873962_C .
:chr1_144873962_C :controls  :chr1_66058513_G .
:chr1_144873962_C :controls  :chrX_134713855_A .
"""

# Write to file

print(buf)
f = open("test.rdf","w") # note this file is shared!
f.write(buf)



### Validate the file - we can do this in python

In [37]:
# pip3 install --user rdflib
# python3 rdfvalidate.py

import rdflib
g = rdflib.Graph()

result = g.parse('test.rdf', format='n3')
print(len(g))
for stmt in g:
    print(stmt)

# You should see something here...

### Here we upload the data to the database

In [38]:
# Equivalent of curl -T $rdf -H 'Content-Type: application/x-turtle' $uri
import requests

myname="pjotr" # change to your username
rdf="test.rdf" # shared file?!
uri="http://zero.borm.org:8000/data/http://biobeat.org/"+myname+"/"+rdf
headers = {'content-type': 'application/x-turtle'}

f = open(rdf)
buf = f.read()
r = requests.put(uri, data=buf, headers=headers)
print(uri)
print(r) # should give a 201


http://zero.borm.org:8000/data/http://biobeat.org/pjotr/test.rdf
<Response [201]>


### Let's check what is in 4store


In [53]:
! lynx --dump zero.borm.org:8000/status/size/

                       SPARQL httpd server status - size

KB pythoncourse

   Segment # quads (s) quads (sr) models resources
   0         663       +0         1      697
   1         755       +0         1      757
     Total   1418      +0         1      1454


### And do a first SPARQL query


In [51]:

end_point = "http://zero.borm.org:8000/sparql/"

query = """
PREFIX : <http://pjotr.me/rdf/pjotr/ns#>
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?s WHERE {
    ?s rdf:type ?o .
} LIMIT 10
"""

r = requests.post(end_point, data={ "query": query, "output": "text" })
# print r.url

print(r.text)


?s
<http://pjotr.me/rdf/ns#chr7_100552867_C>
<http://pjotr.me/rdf/ns#chr17_9515777_A>
<http://pjotr.me/rdf/ns#chr2_32667182_C>
<http://pjotr.me/rdf/ns#chr7_100551849_T>
<http://pjotr.me/rdf/ns#chr7_22202052_A>
<http://pjotr.me/rdf/ns#chr2_112705157_C>
<http://pjotr.me/rdf/ns#chr10_126691608_T>
<http://pjotr.me/rdf/ns#chr1_60466814_C>
<http://pjotr.me/rdf/ns#chr20_29628431_T>
<http://pjotr.me/rdf/ns#chr3_62478014_A>

